In [ ]:
import json
import gradio as gr
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import string
from datetime import datetime
import random

# Download required NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

class CustomerSupportBot:
    def __init__(self, data_path):
        self.data_path = data_path
        self.questions = []
        self.answers = []
        self.vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
        self.question_vectors = None
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        self.chat_history = []

        # Load and process data
        self.load_data()
        self.preprocess_questions()

    def load_data(self):
        """Load FAQ data from JSON file"""
        try:
            with open(self.data_path, 'r', encoding='utf-8') as file:
                for line in file:
                    if line.strip():
                        data = json.loads(line.strip())
                        self.questions.append(data['question'])
                        self.answers.append(data['answer'])
            print(f"Loaded {len(self.questions)} FAQ pairs")
        except Exception as e:
            print(f"Error loading data: {e}")
            # Fallback with sample data
            self.questions = ["How can I create an account?", "What payment methods do you accept?"]
            self.answers = ["Click 'Sign Up' button and follow instructions.", "We accept credit cards, debit cards, and PayPal."]

    def preprocess_text(self, text):
        """Clean and preprocess text"""
        # Convert to lowercase
        text = text.lower()

        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()

        # Tokenize
        tokens = word_tokenize(text)

        # Remove stopwords and lemmatize
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens if token not in self.stop_words]

        return ' '.join(tokens)

    def preprocess_questions(self):
        """Preprocess all questions and create TF-IDF vectors"""
        processed_questions = [self.preprocess_text(q) for q in self.questions]
        self.question_vectors = self.vectorizer.fit_transform(processed_questions)

    def find_best_match(self, user_query, threshold=0.3):
        """Find the best matching question using cosine similarity"""
        processed_query = self.preprocess_text(user_query)
        query_vector = self.vectorizer.transform([processed_query])

        # Calculate similarities
        similarities = cosine_similarity(query_vector, self.question_vectors)[0]

        # Find best match
        best_match_idx = np.argmax(similarities)
        best_similarity = similarities[best_match_idx]

        if best_similarity > threshold:
            return self.answers[best_match_idx], best_similarity, self.questions[best_match_idx]
        else:
            return None, best_similarity, None

    def get_response(self, user_input):
        """Generate response for user input"""
        if not user_input.strip():
            return "Please ask me a question about our products or services!"

        # Find best matching answer
        answer, similarity, matched_question = self.find_best_match(user_input)

        if answer:
            confidence = f"(Confidence: {similarity:.2f})"
            response = f"{answer}\n\n{confidence}"
            if similarity < 0.6:
                response += "\n\nIf this doesn't fully answer your question, please try rephrasing it or contact our support team directly."
        else:
            response = self.get_fallback_response(user_input)

        return response

    def get_fallback_response(self, user_input):
        """Generate fallback response when no good match is found"""
        fallback_responses = [
            "I'm sorry, I couldn't find a specific answer to your question. Please contact our customer support team for personalized assistance.",
            "I don't have information about that specific topic. Our customer support team would be happy to help you with more detailed assistance.",
            "That's a great question! For the most accurate answer, I'd recommend reaching out to our customer support team directly.",
            "I'm still learning about that topic. Please contact our support team for immediate assistance with your inquiry."
        ]

        base_response = random.choice(fallback_responses)

        # Add helpful suggestions based on keywords
        suggestions = []
        keywords = user_input.lower()

        if any(word in keywords for word in ['order', 'shipping', 'delivery']):
            suggestions.append("• You can track your order in the 'Order History' section of your account")
        if any(word in keywords for word in ['return', 'refund']):
            suggestions.append("• Check our return policy for detailed information about returns and refunds")
        if any(word in keywords for word in ['account', 'login', 'password']):
            suggestions.append("• Try using the 'Forgot Password' link if you're having login issues")
        if any(word in keywords for word in ['payment', 'card', 'billing']):
            suggestions.append("• We accept major credit cards, debit cards, and PayPal")

        if suggestions:
            base_response += "\n\nHere are some quick tips that might help:\n" + "\n".join(suggestions)

        base_response += "\n\n📞 Contact our support team at support@eyastore.com"

        return base_response

# Initialize the chatbot
bot = CustomerSupportBot('/content/train_expanded.json')

def chatbot_response(message, history):
    """Generate chatbot response for Gradio interface"""
    if not message.strip():
        return "Please ask me a question!"

    response = bot.get_response(message)
    return response

def clear_chat():
    """Clear chat history"""
    return [], ""

# Create Gradio interface
def create_interface():
    with gr.Blocks(
        title="Customer Support Chatbot",
        theme=gr.themes.Soft(),
        css="""
        .gradio-container {
            max-width: 1200px !important;
            margin: 0 auto !important;
        }
        .chat-container {
            height: 500px !important;
        }
        .header {
            text-align: center;
            padding: 20px;
            background: linear-gradient(135deg, #1a202c 0%, #2d3748 100%);
            color: white;
            border-radius: 10px;
            margin-bottom: 20px;
            box-shadow: 0 4px 12px rgba(0,0,0,0.2);
        }
        .stats-container {
            display: flex;
            justify-content: space-around;
            margin: 20px 0;
            padding: 15px;
            background: #f8f9fa;
            border-radius: 10px;
        }
        .stat-item {
            text-align: center;
            padding: 10px;
        }
        .stat-number {
            font-size: 24px;
            font-weight: bold;
            color: #667eea;
        }
        .stat-label {
            font-size: 14px;
            color: #666;
        }
        .popular-questions {
            background: linear-gradient(135deg, #2d3748 0%, #4a5568 100%) !important;
            color: white !important;
            border-radius: 12px;
            padding: 20px;
            margin-bottom: 20px;
            box-shadow: 0 4px 12px rgba(0,0,0,0.15);
        }
        .popular-questions h3 {
            color: #ffd700 !important;
            font-weight: bold;
            margin-bottom: 15px;
            font-size: 18px;
        }
        .popular-questions ul {
            list-style: none;
            padding: 0;
        }
        .popular-questions li {
            margin: 12px 0;
            padding: 12px 15px;
            background: rgba(255,255,255,0.1) !important;
            border-radius: 8px;
            cursor: pointer;
            transition: all 0.3s ease;
            color: #ffffff !important;
            font-weight: 500;
            border: 1px solid rgba(255,255,255,0.2);
        }
        .popular-questions li:hover {
            background: rgba(255,255,255,0.2) !important;
            transform: translateY(-2px);
            box-shadow: 0 4px 8px rgba(0,0,0,0.2);
        }
        .tips-section {
            background: linear-gradient(135deg, #1a365d 0%, #2c5282 100%) !important;
            color: white !important;
            border-radius: 12px;
            padding: 20px;
            box-shadow: 0 4px 12px rgba(0,0,0,0.15);
        }
        .tips-section h3 {
            color: #63b3ed !important;
            font-weight: bold;
            margin-bottom: 15px;
            font-size: 18px;
        }
        .tips-section ul {
            color: #e2e8f0 !important;
            font-size: 14px;
            line-height: 1.8;
        }
        .tips-section li {
            margin: 8px 0;
            padding-left: 5px;
        }
        .tips-section p {
            color: #e2e8f0 !important;
            font-size: 14px;
            line-height: 1.6;
        }
        .tips-section strong {
            color: #90cdf4 !important;
            font-weight: 600;
        }
        """
    ) as demo:

        # Header
        gr.HTML("""
        <div class="header">
            <h1>🤖 Customer Support Chatbot</h1>
            <p>Ask me anything about our products, services, orders, returns, and more!</p>
        </div>
        """)

        # Stats
        gr.HTML(f"""
        <div class="stats-container">
            <div class="stat-item">
                <div class="stat-number">{len(bot.questions)}</div>
                <div class="stat-label">FAQ Topics</div>
            </div>
            <div class="stat-item">
                <div class="stat-number">24/7</div>
                <div class="stat-label">Available</div>
            </div>
            <div class="stat-item">
                <div class="stat-number">< 1s</div>
                <div class="stat-label">Response Time</div>
            </div>
        </div>
        """)

        with gr.Row():
            with gr.Column(scale=4):
                # Chat interface
                chatbot = gr.Chatbot(
                    height=450,
                    label="Customer Support Chat",
                    placeholder="Start typing your question...",
                    bubble_full_width=False,
                    avatar_images=("👤", "🤖")
                )

                with gr.Row():
                    msg = gr.Textbox(
                        placeholder="Type your question here... (e.g., 'How can I track my order?')",
                        label="Your Question",
                        lines=2,
                        max_lines=5,
                        scale=4
                    )
                    send_btn = gr.Button("Send 📤", variant="primary", scale=1)



            with gr.Column(scale=1):
                # Quick help section
                gr.HTML("""
                <div class="popular-questions">
                    <h3>🔥 Popular Questions</h3>
                    <ul>
                        <li>📦 How to track my order?</li>
                        <li>🔄 What's your return policy?</li>
                        <li>💳 Payment methods accepted?</li>
                        <li>🚚 Shipping information</li>
                        <li>👤 Create an account</li>
                    </ul>
                </div>
                """)

                gr.HTML("""
                <div class="tips-section">
                    <h3>💡 Tips for Better Results</h3>
                    <ul>
                        <li>Be specific in your questions</li>
                        <li>Use keywords like "order", "shipping", "return"</li>
                        <li>Ask one question at a time</li>
                        <li>Check spelling for better matching</li>
                    </ul>
                </div>
                """)

        # Event handlers
        def respond(message, history):
            response = chatbot_response(message, history)
            history.append((message, response))
            return history, ""

        # Bind events
        send_btn.click(respond, inputs=[msg, chatbot], outputs=[chatbot, msg])
        msg.submit(respond, inputs=[msg, chatbot], outputs=[chatbot, msg])

        # Welcome message
        demo.load(
            lambda: [(None, "👋 Hello! I'm your customer support assistant. I can help you with questions about orders, shipping, returns, payments, account management, and more. What can I help you with today?")],
            outputs=chatbot
        )

    return demo

# Launch the interface
if __name__ == "__main__":
    demo = create_interface()
    demo.launch(
        share=True,
        debug=True,
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True
    )

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loaded 200 FAQ pairs
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1de763bc200e0c5799.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
